In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

import nest
nest.ResetKernel()


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.7.0
 Built: Mar  4 2025 17:27:39

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [2]:
# Add the src directory to the Python path if needed
src_path = os.path.abspath(os.path.join(os.getcwd(), '../src'))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Import NeuroWorkflow components
from neuroworkflow import WorkflowBuilder
from neuroworkflow.nodes.network.SNNbuilder_SingleNeuron import SNNbuilder_SingleNeuron
from neuroworkflow.nodes.network.SNNbuilder_Population import SNNbuilder_Population

In [3]:
# Create a Single neuron:

# Create an instance of the node SNNbuilder_SingleNeuron
my_neuron = SNNbuilder_SingleNeuron("pyramidal neuron")
    
# Configure the neuron with biological parameters
my_neuron.configure(
    # Biological identification
    name="Pyramidal Neuron L2/3",
    acronym="PyL23",
    model_type="point_process",
    cell_class="excitatory",
        
    # Signaling properties
    neurotransmitter_types=["AMPA", "NMDA"],
    psp_amplitudes={"AMPA": 0.8, "NMDA": 0.4},
    rise_times={"AMPA": 2.0, "NMDA": 15.0},
        
    # Morphological properties
    dendrite_extent=300.0,  # μm
    dendrite_diameter=2.5,  # μm
        
    # Activity properties
    firing_rate_resting=[2.0, 8.0],    # Hz
    firing_rate_active=[15.0, 40.0],   # Hz
    firing_rate_maximum=[60.0, 120.0], # Hz
        
    # NEST model parameters
    nest_model="iaf_psc_alpha",
    nest_parameters={
        "V_th": -50.0,      # Threshold potential (mV)
        "C_m": 200.0,       # Membrane capacitance (pF)
        "tau_m": 15.0,      # Membrane time constant (ms)
        "V_reset": -70.0,   # Reset potential (mV)
        "t_ref": 2.0        # Refractory period (ms)
    },
        
    # Execution options
    execution_mode="both",  # Execute and generate script
    script_format="both"    # Python and notebook formats
    )
    
print(f"Neuron: {my_neuron._parameters['name']} ({my_neuron._parameters['acronym']})")
print(f"Neuron: {my_neuron._parameters}")

Neuron: Pyramidal Neuron L2/3 (PyL23)
Neuron: {'name': 'Pyramidal Neuron L2/3', 'acronym': 'PyL23', 'model_type': 'point_process', 'cell_class': 'excitatory', 'neurotransmitter_types': ['AMPA', 'NMDA'], 'psp_amplitudes': {'AMPA': 0.8, 'NMDA': 0.4}, 'rise_times': {'AMPA': 2.0, 'NMDA': 15.0}, 'dendrite_extent': 300.0, 'dendrite_diameter': 2.5, 'firing_rate_resting': [2.0, 8.0], 'firing_rate_active': [15.0, 40.0], 'firing_rate_maximum': [60.0, 120.0], 'firing_rate_disease': [0.1, 2.0], 'nest_model': 'iaf_psc_alpha', 'nest_parameters': {'V_th': -50.0, 'C_m': 200.0, 'tau_m': 15.0, 'V_reset': -70.0, 't_ref': 2.0}, 'template_suffix': '_custom', 'execution_mode': 'both', 'script_format': 'both'}


In [4]:
my_neuron.get_info()

{'name': 'pyramidal neuron',
 'type': 'single_neuron_builder',
 'description': 'Build a single neuron with biological and NEST parameters',
 'parameters': {'name': 'Pyramidal Neuron L2/3',
  'acronym': 'PyL23',
  'model_type': 'point_process',
  'cell_class': 'excitatory',
  'neurotransmitter_types': ['AMPA', 'NMDA'],
  'psp_amplitudes': {'AMPA': 0.8, 'NMDA': 0.4},
  'rise_times': {'AMPA': 2.0, 'NMDA': 15.0},
  'dendrite_extent': 300.0,
  'dendrite_diameter': 2.5,
  'firing_rate_resting': [2.0, 8.0],
  'firing_rate_active': [15.0, 40.0],
  'firing_rate_maximum': [60.0, 120.0],
  'firing_rate_disease': [0.1, 2.0],
  'nest_model': 'iaf_psc_alpha',
  'nest_parameters': {'V_th': -50.0,
   'C_m': 200.0,
   'tau_m': 15.0,
   'V_reset': -70.0,
   't_ref': 2.0},
  'template_suffix': '_custom',
  'execution_mode': 'both',
  'script_format': 'both'},
 'optimizable_parameters': {'dendrite_extent': {'optimizable': True,
   'range': [50.0, 500.0],
   'constraints': {'min': 10.0, 'max': 1000.0}},
  

In [5]:
my_neuron._parameters

{'name': 'Pyramidal Neuron L2/3',
 'acronym': 'PyL23',
 'model_type': 'point_process',
 'cell_class': 'excitatory',
 'neurotransmitter_types': ['AMPA', 'NMDA'],
 'psp_amplitudes': {'AMPA': 0.8, 'NMDA': 0.4},
 'rise_times': {'AMPA': 2.0, 'NMDA': 15.0},
 'dendrite_extent': 300.0,
 'dendrite_diameter': 2.5,
 'firing_rate_resting': [2.0, 8.0],
 'firing_rate_active': [15.0, 40.0],
 'firing_rate_maximum': [60.0, 120.0],
 'firing_rate_disease': [0.1, 2.0],
 'nest_model': 'iaf_psc_alpha',
 'nest_parameters': {'V_th': -50.0,
  'C_m': 200.0,
  'tau_m': 15.0,
  'V_reset': -70.0,
  't_ref': 2.0},
 'template_suffix': '_custom',
 'execution_mode': 'both',
 'script_format': 'both'}

In [6]:
# Create a population 
my_pop = SNNbuilder_Population('cortical_population')
    
# Configure the population parameters
my_pop.configure(
    name='Layer 2/3 Pyramidal Neurons',
    acronym='L23Pyr',
    brain_region='Primary Visual Cortex',
    population_size= 500,
    tissue_reference={ #this is a free format dictionary
        'density_per_mm3': 25000,
        'tissue_volume_mm3': 0.2,
        'reference_study': 'Schüz & Palm (1989)',
        'species': 'human',
        'estimation_method': 'stereological counting'
    },
    spatial_dimensions='3D',
    spatial_bounds={
        'x': (0.0, 500.0),
        'y': (0.0, 500.0),
        'z': (0.0, 300.0)
    },
    model_type='point_process',
    # Execution options
    execution_mode="both",  # Execute and generate script
    script_format="both"    # Python and notebook formats
    )
print(f"Population: {my_pop._parameters}")

Population: {'name': 'Layer 2/3 Pyramidal Neurons', 'acronym': 'L23Pyr', 'brain_region': 'Primary Visual Cortex', 'model_type': 'point_process', 'population_size': 500, 'tissue_reference': {'density_per_mm3': 25000, 'tissue_volume_mm3': 0.2, 'reference_study': 'Schüz & Palm (1989)', 'species': 'human', 'estimation_method': 'stereological counting'}, 'spatial_dimensions': '3D', 'spatial_bounds': {'x': (0.0, 500.0), 'y': (0.0, 500.0), 'z': (0.0, 300.0)}, 'custom_positions': None, 'mean_firing_rate': None, 'execution_mode': 'both', 'script_format': 'both'}


In [7]:
my_pop._parameters

{'name': 'Layer 2/3 Pyramidal Neurons',
 'acronym': 'L23Pyr',
 'brain_region': 'Primary Visual Cortex',
 'model_type': 'point_process',
 'population_size': 500,
 'tissue_reference': {'density_per_mm3': 25000,
  'tissue_volume_mm3': 0.2,
  'reference_study': 'Schüz & Palm (1989)',
  'species': 'human',
  'estimation_method': 'stereological counting'},
 'spatial_dimensions': '3D',
 'spatial_bounds': {'x': (0.0, 500.0), 'y': (0.0, 500.0), 'z': (0.0, 300.0)},
 'custom_positions': None,
 'mean_firing_rate': None,
 'execution_mode': 'both',
 'script_format': 'both'}

In [8]:
# Create a workflow builder
workflow = WorkflowBuilder("Simple population of spiking neurons")

# Add nodes to the workflow
workflow.add_node(my_neuron)
workflow.add_node(my_pop)

In [9]:
# Get port names from any node instance

my_neuron_output_ports = list(my_neuron.get_info()['output_ports'].keys())  
print(f"Neuron output ports: {my_neuron_output_ports}")

print(f"Population input ports: {my_pop.get_info()['input_ports'].keys()}")    

Neuron output ports: ['nest_neuron', 'nest_model_name', 'python_script', 'notebook_cell', 'neuron_metadata', 'biological_properties', 'nest_properties']
Population input ports: dict_keys(['nest_neuron', 'nest_model_name', 'python_script', 'notebook_cell', 'neuron_metadata', 'biological_properties', 'nest_properties'])


In [10]:
# Connect the nodes by ports

workflow.connect(
    my_neuron.name, my_neuron_output_ports[1], 
    my_pop.name, "nest_model_name"
)
workflow.connect(
    my_neuron.name, my_neuron_output_ports[-1], 
    my_pop.name, "nest_properties"
)
workflow.connect(
    "pyramidal neuron", "python_script", 
    "cortical_population", "python_script"
)

In [11]:
workflow.get_connection_count()

3

In [12]:
# Build the workflow
workflow_ready = workflow.build()

# Print workflow information
print(workflow_ready)

Workflow: Simple population of spiking neurons
Nodes:
  pyramidal neuron
  cortical_population
Connections:
  pyramidal neuron.nest_model_name -> cortical_population.nest_model_name
  pyramidal neuron.nest_properties -> cortical_population.nest_properties
  pyramidal neuron.python_script -> cortical_population.python_script


In [13]:
# Execute the workflow
print("Executing workflow...")
success = workflow_ready.execute()

if success:
    print("\nWorkflow execution completed successfully!")
else:
    print("\nWorkflow execution failed!")

Executing workflow...
Executing node: pyramidal neuron
[pyramidal neuron] Validating neuron parameters...
[pyramidal neuron] Warning: Could not verify NEST model availability: 'tuple' object is not callable
[pyramidal neuron] Parameter validation completed successfully
[pyramidal neuron] Creating NEST neuron...
[pyramidal neuron] Copying model: iaf_psc_alpha -> iaf_psc_alpha_custom
[pyramidal neuron] Setting defaults: {'V_th': -50.0, 'C_m': 200.0, 'tau_m': 15.0, 'V_reset': -70.0, 't_ref': 2.0}
[pyramidal neuron] Creating neuron from template: iaf_psc_alpha_custom
[pyramidal neuron] NEST neuron created successfully: NodeCollection(metadata=None, model=iaf_psc_alpha_custom, size=1, first=1)
[pyramidal neuron] Generating Python script...
[pyramidal neuron] Python script generated (24 lines)
[pyramidal neuron] Generating Jupyter notebook cell...
[pyramidal neuron] Generating Python script...
[pyramidal neuron] Python script generated (24 lines)
[pyramidal neuron] Notebook cell generated
[p

In [14]:
print(my_neuron.get_output_port("python_script").value)

# Single Neuron: Pyramidal Neuron L2/3 (PyL23)
# Generated by NeuroWorkflow SingleNeuronBuilderNode
# Model Type: point_process
# Cell Class: excitatory

import nest

# === BIOLOGICAL PROPERTIES ===
# Dendrite extent: 300.0 μm
# Dendrite diameter: 2.5 μm
# Neurotransmitters: ['AMPA', 'NMDA']
# PSP amplitudes: {'AMPA': 0.8, 'NMDA': 0.4}
# Rise times: {'AMPA': 2.0, 'NMDA': 15.0}
# Firing rates - Resting: [2.0, 8.0] Hz
# Firing rates - Active: [15.0, 40.0] Hz
# Firing rates - Maximum: [60.0, 120.0] Hz

# === NEST MODEL CREATION ===
nest.CopyModel("iaf_psc_alpha", "iaf_psc_alpha_custom")
nest.SetDefaults("iaf_psc_alpha_custom", {
                        "V_th": -50.0,
                        "C_m": 200.0,
                        "tau_m": 15.0,
                        "V_reset": -70.0,
                        "t_ref": 2.0
                    })
pyl23 = nest.Create("iaf_psc_alpha_custom", 1)

print(f"Created neuron: {pyl23}")
print(f"Model template: iaf_psc_alpha_custom")


In [15]:
print(my_neuron.get_output_port("notebook_cell").value)

# Markdown Cell
```markdown
## Neuron: Pyramidal Neuron L2/3 (PyL23)

**Biological Properties:**
- Model Type: point_process
- Cell Class: excitatory
- Dendrite Extent: 300.0 μm
- Dendrite Diameter: 2.5 μm
- Neurotransmitters: AMPA, NMDA

**NEST Model:**
- Base Model: iaf_psc_alpha
- Custom Parameters: {'V_th': -50.0, 'C_m': 200.0, 'tau_m': 15.0, 'V_reset': -70.0, 't_ref': 2.0}
```

# Code Cell
```python
# Single Neuron: Pyramidal Neuron L2/3 (PyL23)
# Generated by NeuroWorkflow SingleNeuronBuilderNode
# Model Type: point_process
# Cell Class: excitatory

import nest

# === BIOLOGICAL PROPERTIES ===
# Dendrite extent: 300.0 μm
# Dendrite diameter: 2.5 μm
# Neurotransmitters: ['AMPA', 'NMDA']
# PSP amplitudes: {'AMPA': 0.8, 'NMDA': 0.4}
# Rise times: {'AMPA': 2.0, 'NMDA': 15.0}
# Firing rates - Resting: [2.0, 8.0] Hz
# Firing rates - Active: [15.0, 40.0] Hz
# Firing rates - Maximum: [60.0, 120.0] Hz

# === NEST MODEL CREATION ===
nest.CopyModel("iaf_psc_alpha", "iaf_psc_alpha_custom")
n

In [16]:
print(my_pop.get_output_port("python_script").value)

# Population: Layer 2/3 Pyramidal Neurons (L23Pyr)
# Generated by NeuroWorkflow PopulationBuilderNode
# Brain Region: Primary Visual Cortex
# Population Size: 500 neurons (single hemisphere)
# Model Type: point_process
# Spatial: 3D

import numpy as np

# === POPULATION BIOLOGICAL CONTEXT ===
# Density: 25000 neurons/mm³
# Tissue volume: 0.2 mm³
# Reference: Schüz & Palm (1989)
# Species: human
# Estimation method: stereological counting
# Spatial bounds: {'x': (0.0, 500.0), 'y': (0.0, 500.0), 'z': (0.0, 300.0)}

# === SPATIAL POSITIONS ===
# 3D positions for 500 neurons
# Spatial bounds: {'x': (0.0, 500.0), 'y': (0.0, 500.0), 'z': (0.0, 300.0)}

# Generate spatial positions
positions = np.zeros((500, 3))
positions[:, 0] = np.random.uniform(0.0, 500.0, 500)  # x coordinates
positions[:, 1] = np.random.uniform(0.0, 500.0, 500)  # y coordinates
positions[:, 2] = np.random.uniform(0.0, 300.0, 500)  # z coordinates

# === POPULATION CREATION ===
# Using neuron model from SingleNeuronBuilde

In [17]:
print(my_pop.get_output_port("notebook_cell").value)

# Population: Layer 2/3 Pyramidal Neurons
# 500 neurons, 3D

# Create spatial positions
positions = np.random.uniform([0.0, 0.0, 0.0], [500.0, 500.0, 300.0], (500, 3))
nest_positions = nest.spatial.free(positions.tolist())
l23pyr = nest.Create('iaf_psc_alpha_custom', 500, positions=nest_positions)
neuron_status = nest.GetStatus(l23pyr[0])
print(f"Status of first neuron: {neuron_status}")


In [18]:
execution_sequence = workflow_ready.get_execution_sequence()
print(execution_sequence)

{'workflow_name': 'Simple population of spiking neurons', 'execution_timestamp': '2025-09-18 20:59:59', 'total_nodes': 2, 'execution_sequence': [{'node_name': 'pyramidal neuron', 'node_instance': <neuroworkflow.nodes.network.SNNbuilder_SingleNeuron.SNNbuilder_SingleNeuron object at 0x126e41010>, 'node_type': 'single_neuron_builder', 'execution_order': 0, 'timestamp': 1758196796.307792, 'duration': 0.0018138885498046875, 'success': True, 'has_python_script': True, 'has_notebook_cell': True, 'output_ports': ['nest_neuron', 'nest_model_name', 'python_script', 'notebook_cell', 'neuron_metadata', 'biological_properties', 'nest_properties']}, {'node_name': 'cortical_population', 'node_instance': <neuroworkflow.nodes.network.SNNbuilder_Population.SNNbuilder_Population object at 0x126e41730>, 'node_type': 'population_builder', 'execution_order': 1, 'timestamp': 1758196796.309626, 'duration': 0.008254051208496094, 'success': True, 'has_python_script': True, 'has_notebook_cell': True, 'output_po

In [19]:
from neuroworkflow.utils import export_workflow_scripts

In [20]:
files = export_workflow_scripts(
        execution_sequence=execution_sequence,
        output_dir="./",           # Custom directory
        export_python=True,                              # Export Python script
        export_notebook=True,                            # Export Jupyter notebook
        filename_base="code_gen_test",                   # Custom filename base
        deduplicate_imports=True,                        # Remove duplicate imports
        add_metadata=True                                # Add workflow metadata
)

Processing workflow: Simple population of spiking neurons
Total nodes in execution order: 2
Processing node: pyramidal neuron (has_python: True, has_notebook: True)
  → Collected Python script (980 chars)
  → Collected notebook cell (1392 chars) + python script (980 chars)
Processing node: cortical_population (has_python: True, has_notebook: True)
  → Collected Python script (1566 chars)
  → Collected notebook cell (390 chars) + python script (1566 chars)

Summary:
  Python script fragments: 2
  Notebook cell fragments: 2
  ✅ Python script exported to: ./code_gen_test.py
  ✅ Jupyter notebook exported to: ./code_gen_test.ipynb


In [21]:
my_neuron.get_output_port('notebook_cell').value

'# Markdown Cell\n```markdown\n## Neuron: Pyramidal Neuron L2/3 (PyL23)\n\n**Biological Properties:**\n- Model Type: point_process\n- Cell Class: excitatory\n- Dendrite Extent: 300.0 μm\n- Dendrite Diameter: 2.5 μm\n- Neurotransmitters: AMPA, NMDA\n\n**NEST Model:**\n- Base Model: iaf_psc_alpha\n- Custom Parameters: {\'V_th\': -50.0, \'C_m\': 200.0, \'tau_m\': 15.0, \'V_reset\': -70.0, \'t_ref\': 2.0}\n```\n\n# Code Cell\n```python\n# Single Neuron: Pyramidal Neuron L2/3 (PyL23)\n# Generated by NeuroWorkflow SingleNeuronBuilderNode\n# Model Type: point_process\n# Cell Class: excitatory\n\nimport nest\n\n# === BIOLOGICAL PROPERTIES ===\n# Dendrite extent: 300.0 μm\n# Dendrite diameter: 2.5 μm\n# Neurotransmitters: [\'AMPA\', \'NMDA\']\n# PSP amplitudes: {\'AMPA\': 0.8, \'NMDA\': 0.4}\n# Rise times: {\'AMPA\': 2.0, \'NMDA\': 15.0}\n# Firing rates - Resting: [2.0, 8.0] Hz\n# Firing rates - Active: [15.0, 40.0] Hz\n# Firing rates - Maximum: [60.0, 120.0] Hz\n\n# === NEST MODEL CREATION ===

In [22]:
my_pop.get_output_port('notebook_cell').value

'# Population: Layer 2/3 Pyramidal Neurons\n# 500 neurons, 3D\n\n# Create spatial positions\npositions = np.random.uniform([0.0, 0.0, 0.0], [500.0, 500.0, 300.0], (500, 3))\nnest_positions = nest.spatial.free(positions.tolist())\nl23pyr = nest.Create(\'iaf_psc_alpha_custom\', 500, positions=nest_positions)\nneuron_status = nest.GetStatus(l23pyr[0])\nprint(f"Status of first neuron: {neuron_status}")'